Импортируем библиотеки

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import time
from datetime import datetime
import random

In [33]:
import string

In [34]:
punctuation = string.punctuation

In [2]:
import re

In [3]:
import nltk

In [4]:
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
!pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 858.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.6 MB/s eta 0:00:00


In [6]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

In [7]:
import pandas as pd

In [8]:
import os

In [9]:
session = requests.session()

Функция добычи ссылок

In [10]:
def href_list(link):
    lst = []
    response = session.get(link)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    authors = soup.find_all('a', href=True)
    for author in authors:
        lst.append(author['href'])
    return lst

Функция добычи текста

In [11]:
def get_text(link):
    response = session.get(link)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    title = soup.find('title').text.split('. ')
    if len(title) > 1:
        author = title[0]
        book_title = title[1]
        title_string = '. '.join(title)
        text = soup.text
        text_list = text.split(title_string)
        text = text_list[-1]
        idxs = list(re.finditer('---', text))
        if len(idxs) > 0:
            text = text[idxs[-1].end():]
        text = text.replace('\n', ' ')
        return author, book_title, text
    else:
        print(title)
        return -1, -1, -1

Собираем список авторов детективов

In [12]:
author_list = href_list('http://lib.ru/RUSS_DETEKTIW/')

In [13]:
new_author_list = []
for author in author_list:
    f  = True
    author_1 = author.replace('/', '')
    alpha = '_QWERTYUIOPASDFGHJKLZXCVBNM'
    for let in author_1:
        if let not in alpha:
            f = False
            break
    if f:
        new_author_list.append(author)

In [14]:
new_author_list = new_author_list[8:]

In [15]:
base_link = 'http://lib.ru/RUSS_DETEKTIW/'

In [16]:
dict_authors = dict()

Собираем список книг для всех авторов

In [17]:
for author in new_author_list:
    link = base_link + '/' + author
    dict_authors[author] = []
    h_list = href_list(link)
    for href in h_list:
        if href[-4:] == '.txt':
            if '/' not in href:
                dict_authors[author].append(href)

In [ ]:
k = 0
for author in dict_authors:
    print(dict_authors[author])
    k += len(dict_authors[author])
print(k)  # количество текстов

In [45]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Функция, проверяющая слово ли токен (исключает пунктуацию)

In [59]:
def is_alpha_1(word):
    if len(word) < 2:
        if not word.isalpha():
            return False
    else:
        for i in range(1, len(word)):
            if not word[i].isalpha() and not word[i-1].isalpha():
                return False
    return True

Функция, собирающая файлы с текстами и список слов

In [60]:
def main_function(author_list, dict_authors, base_link, start, end):
    global NMB_TEXT
    global WORDS_LIST
    for author in author_list[start:end]:  # taking authors from the start to the end
        list_texts = dict_authors[author] # list of all author's titles
        for text in list_texts:
            sents_list = [] #list of all dicts with sentences and its numbers of one text
            link = base_link + '/' + author + text
            book_author, book_title, text = get_text(link)
            if book_author != -1:
                nmb_words = 0
                nmb_sent = 1
                for sent in sent_tokenize(text, language='russian'):
                    words = word_tokenize(sent)
                    words = [w for w in words if is_alpha_1(w)]
                    if words != []:
                        nmb_word = 1
                        for word in words:
                            ana = morph.parse(word)[0]
                            WORDS_LIST.append({'word': ana.word, 'lemma': ana.normal_form, 'pos': ana.tag.POS, 'nmb_text': NMB_TEXT, 'nmb_sent': nmb_sent, 'nmb_word': nmb_word})
                            nmb_word += 1
                        nmb_words += nmb_word
                        sents_list.append({'sentence': sent, 'nmb_sent': nmb_sent, 'author':book_author, 'title':book_title})
                        nmb_sent += 1
                        if nmb_words >= 100:
                            break
                sent_df = pd.DataFrame(sents_list)
                file_name = 'text' + str(NMB_TEXT) +'.csv'
                sent_df.to_csv(file_name, encoding='utf-8')
                NMB_TEXT += 1
            else:
                continue


In [36]:
word_tokenize('мама мыла-мыла, раму')

['мама', 'мыла-мыла', ',', 'раму']

In [49]:
NMB_TEXT

56

In [ ]:
Собираем по частям, чтобы не ломался сайт

In [61]:
WORDS_LIST = []
NMB_TEXT = 1

In [62]:
main_function(new_author_list, dict_authors, base_link, 0, 5)

['/RUSS_DETEKTIW/BAKUNIN/leviafan.txt']
['Марианна Баконина: об авторе']
['Copyright notice']


In [63]:
main_function(new_author_list, dict_authors, base_link, 5, 10)

['Copyright notice']


In [64]:
main_function(new_author_list, dict_authors, base_link, 10, 15)

In [65]:
main_function(new_author_list, dict_authors, base_link, 15, 20)

['/RUSS_DETEKTIW/MIHAJLIK/black.txt']
['Чужаки']


In [66]:
main_function(new_author_list, dict_authors, base_link, 20, 25)

['Copyright notice']
['Евгения Кукаркина: библиография']


Записываем слова в файл

In [67]:
word_df = pd.DataFrame(WORDS_LIST)
file_name = 'words.csv'
word_df
word_df.to_csv(file_name, encoding='utf-8')

Загружаем файлы на компьютер

In [30]:
directory_path = '/content/'
directory_files = os.listdir(directory_path)
print(sorted(directory_files))
print(len(directory_files))

['.config', 'sample_data', 'text1.csv', 'text10.csv', 'text100.csv', 'text101.csv', 'text102.csv', 'text103.csv', 'text104.csv', 'text105.csv', 'text106.csv', 'text107.csv', 'text108.csv', 'text109.csv', 'text11.csv', 'text110.csv', 'text111.csv', 'text112.csv', 'text113.csv', 'text114.csv', 'text115.csv', 'text116.csv', 'text117.csv', 'text118.csv', 'text119.csv', 'text12.csv', 'text120.csv', 'text121.csv', 'text122.csv', 'text123.csv', 'text13.csv', 'text14.csv', 'text15.csv', 'text16.csv', 'text17.csv', 'text18.csv', 'text19.csv', 'text2.csv', 'text20.csv', 'text21.csv', 'text22.csv', 'text23.csv', 'text24.csv', 'text25.csv', 'text26.csv', 'text27.csv', 'text28.csv', 'text29.csv', 'text3.csv', 'text30.csv', 'text31.csv', 'text32.csv', 'text33.csv', 'text34.csv', 'text35.csv', 'text36.csv', 'text37.csv', 'text38.csv', 'text39.csv', 'text4.csv', 'text40.csv', 'text41.csv', 'text42.csv', 'text43.csv', 'text44.csv', 'text45.csv', 'text46.csv', 'text47.csv', 'text48.csv', 'text49.csv', '

In [31]:
from google.colab import files
for f in sorted(directory_files)[0:10]:
    if f != '.config' and f != 'sample_data' and f != '.ipynb_checkpoints':
        files.download(f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
files.download('words.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
main_function(new_author_list, dict_authors, base_link, 25, 30)

Мусорка (не прям мусорка - но код на этапе его усовершенствования)

In [ ]:
words_list = []
for author in new_author_list[:8]:
    list_texts = dict_authors[author] # list of all author's titles
    nmb_text = 1
    for text in list_texts:
        sents_list = [] #list of all dicts with sentences and its numbers of one text
        link = base_link + '/' + author + text
        book_author, book_title, text = get_text(link)
        if book_author != -1:
            nmb_words = 0
            nmb_sent = 1
            for sent in sent_tokenize(text):
                words = word_tokenize(sent)
                words = [w for w in words if w.isalpha()]
                if words != []:
                    nmb_word = 1
                    for word in words:
                        ana = morph.parse(word)[0]
                        words_list.append({'word': ana.word, 'lemma': ana.normal_form, 'pos': ana.tag.POS, 'nmb_text': nmb_text, 'nmb_sent': nmb_sent, 'nmb_word': nmb_word})
                        nmb_word += 1
                    nmb_words += nmb_word
                    sents_list.append({'sentence': sent, 'nmb_sent': nmb_sent, 'author':book_author, 'title':book_title})
                    nmb_sent += 1
                    if nmb_words >= 100:
                        break
            sent_df = pd.DataFrame(sents_list)
            file_name = 'text' + str(nmb_text) +'.csv'
            sent_df.to_csv(file_name, encoding='utf-8')
            nmb_text += 1
        else:
            continue

['/RUSS_DETEKTIW/BAKUNIN/leviafan.txt']
['Марианна Баконина: об авторе']
['Copyright notice']
['Copyright notice']


In [ ]:
for author in new_author_list[8:16]:
    list_texts = dict_authors[author] # list of all author's titles
    for text in list_texts:
        sents_list = [] #list of all dicts with sentences and its numbers of one text
        link = base_link + '/' + author + text
        book_author, book_title, text = get_text(link)
        if book_author != -1:
            nmb_words = 0
            nmb_sent = 1
            for sent in sent_tokenize(text):
                nmb_word = 1
                words = word_tokenize(sent)
                words = [w for w in words if w.isalpha()]
                for word in words:
                    ana = morph.parse(word)[0]
                    words_list.append({'word': ana.word, 'lemma': ana.normal_form, 'pos': ana.tag.POS, 'nmb_text': nmb_text, 'nmb_sent': nmb_sent, 'nmb_word': nmb_word})
                    nmb_word += 1
                nmb_words += nmb_word
                sents_list.append({'sentence': sent, 'nmb_sent': nmb_sent, 'author':book_author, 'title':book_title})
                nmb_sent += 1
                if nmb_words >= 100:
                    break
            sent_df = pd.DataFrame(sents_list)
            file_name = 'text' + str(nmb_text) +'.csv'
            sent_df.to_csv(file_name, encoding='utf-8')
            nmb_text += 1
        else:
            continue

In [ ]:
for author in new_author_list[16:24]:
    list_texts = dict_authors[author] # list of all author's titles
    for text in list_texts:
        sents_list = [] #list of all dicts with sentences and its numbers of one text
        link = base_link + '/' + author + text
        book_author, book_title, text = get_text(link)
        if book_author != -1:
            nmb_words = 0
            nmb_sent = 1
            for sent in sent_tokenize(text):
                nmb_word = 1
                words = word_tokenize(sent)
                words = [w for w in words if w.isalpha()]
                for word in words:
                    ana = morph.parse(word)[0]
                    words_list.append({'word': ana.word, 'lemma': ana.normal_form, 'pos': ana.tag.POS, 'nmb_text': nmb_text, 'nmb_sent': nmb_sent, 'nmb_word': nmb_word})
                    nmb_word += 1
                nmb_words += nmb_word
                sents_list.append({'sentence': sent, 'nmb_sent': nmb_sent, 'author':book_author, 'title':book_title})
                nmb_sent += 1
                if nmb_words >= 100:
                    break
            sent_df = pd.DataFrame(sents_list)
            file_name = 'text' + str(nmb_text) +'.csv'
            sent_df.to_csv(file_name, encoding='utf-8')
            nmb_text += 1
        else:
            continue

['/RUSS_DETEKTIW/MIHAJLIK/black.txt']
['Чужаки']


In [ ]:
for author in new_author_list[24:32]:
    list_texts = dict_authors[author] # list of all author's titles
    for text in list_texts:
        sents_list = [] #list of all dicts with sentences and its numbers of one text
        link = base_link + '/' + author + text
        book_author, book_title, text = get_text(link)
        if book_author != -1:
            nmb_words = 0
            nmb_sent = 1
            for sent in sent_tokenize(text):
                nmb_word = 1
                words = word_tokenize(sent)
                words = [w for w in words if w.isalpha()]
                for word in words:
                    ana = morph.parse(word)[0]
                    words_list.append({'word': ana.word, 'lemma': ana.normal_form, 'pos': ana.tag.POS, 'nmb_text': nmb_text, 'nmb_sent': nmb_sent, 'nmb_word': nmb_word})
                    nmb_word += 1
                nmb_words += nmb_word
                sents_list.append({'sentence': sent, 'nmb_sent': nmb_sent, 'author':book_author, 'title':book_title})
                nmb_sent += 1
                if nmb_words >= 100:
                    break
            sent_df = pd.DataFrame(sents_list)
            file_name = 'text' + str(nmb_text) +'.csv'
            sent_df.to_csv(file_name, encoding='utf-8')
            nmb_text += 1
        else:
            continue

['Copyright notice']
['Евгения Кукаркина: библиография']


In [ ]:
for author in new_author_list[32:33]:
    list_texts = dict_authors[author] # list of all author's titles
    for text in list_texts:
        sents_list = [] #list of all dicts with sentences and its numbers of one text
        link = base_link + '/' + author + text
        book_author, book_title, text = get_text(link)
        if book_author != -1:
            nmb_words = 0
            nmb_sent = 1
            for sent in sent_tokenize(text):
                nmb_word = 1
                words = word_tokenize(sent)
                words = [w for w in words if w.isalpha()]
                for word in words:
                    ana = morph.parse(word)[0]
                    words_list.append({'word': ana.word, 'lemma': ana.normal_form, 'pos': ana.tag.POS, 'nmb_text': nmb_text, 'nmb_sent': nmb_sent, 'nmb_word': nmb_word})
                    nmb_word += 1
                nmb_words += nmb_word
                sents_list.append({'sentence': sent, 'nmb_sent': nmb_sent, 'author':book_author, 'title':book_title})
                nmb_sent += 1
                if nmb_words >= 100:
                    break
            sent_df = pd.DataFrame(sents_list)
            file_name = 'text' + str(nmb_text) +'.csv'
            sent_df.to_csv(file_name, encoding='utf-8')
            nmb_text += 1
        else:
            continue

['"Слагать из встречных лиц один портрет..."(Интервью с писателем Раулем Мир-Хайдаровым)']
['Кто ничего не умеет, тот не должен ничего хотеть...Интервью Арынгазы Беркинбаева с Раулем Мир-Хайдаровым специально для Казахстана, в год юбилея писателя']


In [ ]:
lst_nmb = [4, 7, 8, 12, 13, 39, 40, 52, 53, 66, 67, 68, 69, 80, 81, 82, 83, 120]

In [ ]:
for i in lst_nmb:
    f = 'text' + str(i) + '.csv'
    files.download(f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('words.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
search = 'себе'
sent = "он написал себе себе домой"
sent_ind = 0
all = []
while sent_ind < 1:
    spis = []
    srch_i = 0
    words = sent.split()
    word_i = 0
    spis_n = []
    while word_i < len(words) and srch_i < len(search.split(' ')):
        if words[word_i] == search.split(' ')[srch_i]:
            spis_n.append(words[word_i])
            if srch_i == len(search.split(' '))-1:
                spis.append(spis_n)
                srch_i = 0
            else:
                srch_i+=1
        else:
            spis_n = []
            srch_i = 0
        word_i+=1
    sent_ind+=1
    if spis!=[]:
        all.append(spis)
print(all)

[[['себе', 'себе'], ['себе', 'себе']]]


In [ ]:
if search_first[0]!='"' and search_first[0]!='"':
    res = []
    search = search_first.split()
    text_id = 1
    while text_id<max(first_df['nmb_text']):
        df = first_df[first_df['nmb_text']==text_id]
        sent_ind = 1
        all = []
        while sent_ind<=max(df['nmb_sent']):
            spis = []
            srch_i = 0
            words = list(df[df['nmb_sent']==sent_ind]['word'].values)
            tags = list(df[df['nmb_sent']==sent_ind]['pos'].values)
            lemmas = list(df[df['nmb_sent']==sent_ind]['lemma'].values)
            word_i = 0
            spis_n = []
            while word_i<len(words) and srch_i<len(search):
                if search[srch_i][0]=='"' and search[srch_i][-1]=='"':
                    if '+' in search[srch_i]:
                        data = search[srch_i].split('+')
                        if data[0]==words[word_i] and data[1]==tags[word_i]:
                            spis_n.append(words[word_i])
                            print(word_i, srch_i, words[word_i], search[srch_i], len(words), sent_ind)
                            srch_i+=1
                    elif words[word_i] == search[srch_i]:
                        spis_n.append(words[word_i])
                        print(word_i, srch_i, words[word_i], search[srch_i], len(words), sent_ind)
                        srch_i+=1
                    elif tags[word_i] == search[srch_i]:
                        spis_n.append(tags[word_i])
                        print(word_i, srch_i, tags[word_i], search[srch_i], len(words), sent_ind)
                        srch_i+=1
                    else:
                        word_i -=len(spis_n)
                        srch_1 = 0
                        spis_n = []
                elif search[srch_i][0]!='"' and search[srch_i][-1]!='"':
                    lem = m.parse(search[srch_i])[0].normal_form
                    if '+' in search[srch_i]:
                        data = search[srch_i].split('+')
                        if m.parse(data[0])[0].normal_form==lemmas[word_i] and data[1]==tags[word_i]:
                            spis_n.append(lemmas[word_i])
                            print(word_i, srch_i, lemmas[word_i], search[srch_i], len(lemmas), sent_ind)
                            srch_i+=1
                    elif lemmas[word_i] == lem:
                        spis_n.append(lemmas[word_i])
                        print(word_i, srch_i, lemmas[word_i], lem, len(words), sent_ind)
                        srch_i+=1
                    elif tags[word_i] == search[srch_i]:
                        spis_n.append(tags[word_i])
                        print(word_i, srch_i, tags[word_i], search[srch_i], len(words), sent_ind)
                        srch_i+=1
                    else:
                        word_i -=len(spis_n)
                        srch_1 = 0
                        spis_n = []

                if spis_n!=[] and len(search) == len(spis_n):
                    spis.append(spis_n)
                    srch_i = 0
                    word_i -=len(spis_n)
                word_i+=1
            # srch_i+=1
            sent_ind+=1
            if spis!=[]:
                all.append(spis)
        text_id+=1
        res.append(all)
        print(res)
elif search_first[0]=='"' and search_first[0]=='"':
    res = []
    search = (search_first[1:-1]).split()
    text_id = 1
    while text_id<max(first_df['nmb_text']):
        df = first_df[first_df['nmb_text']==text_id]
        sent_ind = 0
        all = []
        while sent_ind<max(df['nmb_sent']):
            spis = []
            srch_i = 0
            words = list(df[df['nmb_sent']==sent_ind]['word'].values)
            tags = list(df[df['nmb_sent']==sent_ind]['pos'].values)
            lemmas = list(df[df['nmb_sent']==sent_ind]['lemma'].values)
            word_i = 0
            spis_n = []
            while word_i<len(words) and srch_i<len(search):
                if '+' in search[srch_i]:
                    data = search[srch_i].split('+')
                    if data[0]==words[word_i] and data[1]==tags[word_i]:
                        spis_n.append(words[word_i])
                        print(word_i, srch_i, words[word_i], search[srch_i], len(words), sent_ind)
                        srch_i+=1
                elif words[word_i] == search[srch_i]:
                    spis_n.append(words[word_i])
                    print(word_i, srch_i, words[word_i], search[srch_i], len(words), sent_ind)
                    srch_i+=1
                elif tags[word_i] == search[srch_i]:
                    spis_n.append(tags[word_i])
                    print(word_i, srch_i, tags[word_i], search[srch_i], len(words), sent_ind)
                    srch_i+=1
                else:
                    word_i -=len(spis_n)
                    srch_1 = 0
                    spis_n = []
                if spis_n!=[] and len(search) == len(spis_n):
                    word_i -=len(spis_n)
                    srch_1 = 0
                    spis.append(spis_n)
                    spis_n = []
                word_i+=1
            #srch_i+=1
            sent_ind+=1
        if spis!=[]:
            all.append(spis)
        text_id+=1
    res.append(all)
    print(res)

In [ ]:
texxxt = 'Валерий Горбань. Законодатель - лучший друг угонщика (и другие статьи) СодержаниеFine HTMLPrinted versiontxt(Word,КПК)Lib.ru html Валерий Горбань. Законодатель - лучший друг угонщика (и другие статьи) Обожженные ненавистью'
list(re.finditer(r'Валерий Горбань. Законодатель - лучший друг угонщика (и другие статьи)', texxxt))

[]